# Hey you, Data Scientist! Learn yourself some operations

You've studied machine learning, you're a dataframe master for massaging data, and you can easily pipe that data through scikit-learn. 

You go for a job interview at a SASS company, you're given some raw data and labels and asked to predict churn, and come on - are these guys even trying? You generate the shit out of some features, you overfit the hell out of that multidimensional manifold just so you can back off and show off your knowledge of regularization, and then you put the icing on the cake by cross validating towards a better metric for the business problem than simple accuracy. 

Next.

You roll up on an ecommerce company, and they trick you by basically giving you no features. Just users, products, and clicks? Ha! Nice try, but you know that's a classic recommender system. Boom! Out of nowhere, the ratings matrix is factorized, and you're serving up top-notch recommendations a-plenty.

Next.

Your uncle calls you and asks for a "family favor". He manufactures artisinal dog collars in a Brooklyn warehouse and sells directly to the consumer for cheap by cutting out any middlemen. He doesn't know what the word churn means. He needs to decide how many of his 10 types of dog collars he should build next month. His company's bootstrapped and can never meet the customer demand, so the products always sell out. Each collar in the catlog has a different profit margin, a different minimum build quantity, and there's only a $10K budget. Uncle likes having as many collar types on the shelves as possible, so he plans to produce roughly similar collar quantities (within 30 of each other). Uncle asks you, "How many of each collar should we build to maximize profit?"

Uhhhhh.

You panic and try to apply ML to everything. *Hmmm, profit is a not a label but more a continuous variable. Okay, not a classification problem. Maybe this is a regression problem? Do I even want to predict the profit? Wait, what are my features? Shit, there's like no data...*

And so you write a program to enumerate all possible combinations of dog collar quantities along with the associated profit, and you find the maximum of the list of answers. Your uncle's happy, but you feel like an idiot because you know that this isn't scalable and god help you if you're ever asked this in an interview and there's just *got* to be a better way, right?

Well, there's a better way.

There's a better way! And I feel like nobody talks about it because the Data's not Big, you're not Learning Deep things, and there's nary a chatbot in sight. It's boring, old operations research which was something that I guess your university offered but nobody really knew what it meant. Full disclosure: I still don't really know what it means. I *do* know that the job of the data scientist is to bring value to the company, and having some operations in your toolbelt is quite valuable!

## Problem formulation

Let's try to solve a simple version of Uncle's problem to get a feel for how we think about these things. I am not going to in any way cover how programs actually solve these problems (because I still don't know) but instead how one sets up this problem and asks a solver to solve this in Python.

What was the original question? "How many of each collar should we build to maximize profit?" Maximize profit is the key phrase here. The way to think of optimization problems like this are in terms of the objective function and the constraints. We want to maximize profit, so our objective function will be the total profit from all collars produced (assuming all get sold). If we say that we have a variable, $c_{j}$, which says how many of collar $j$ we will produce, and $p_{j}$ is a constant denoting the profit that we will make on that collar, then our objective function is simply

$$\sum\limits_{j}p_{j}c_{j}$$

We want to maximize this function subject to specific constraints. One constraint is that we only have a \$5K budget. The simplest method of solving these types of problems is to keep all constraints linear. If we have known constants, $w_{j}$ which is the cost in dollars of producing collar $j$, then this constraint could be expressed in mathematical form as

$$\sum\limits_{j}w_{j}c_{j} \leq 5000$$

Isn't this fun? All of these optimization problems consist of figuring out how to define your constraints in terms of *math* which is basically like the ultimate nerd-puzzle. The constraint on figuring out your constraints (see what I did there?) is that you should keep your constraints linear. This can get fairly tricky (read: fun).

Alright, let's figure out the last constraint, and then we can start coding this thing up. If $r_{j}$ is the minimimum run size for each collar, then this is easy:

$$\forall\, j,\, c_{j} >= r_{j} $$

By the way, that means "for all $j$, $c_{j}$ is greater than or equal to 50".

## Math $\Rightarrow$ Code

At Birchbox, I programmed and solved a number of optimization problems, and I had the luxury to use [Gurobi](http://www.gurobi.com/), a state of the art mixed-integer programming solver with API's in many languages. Gurobi is most definitely not free, so I won't use it in this blog post on the off chance that you want to program along. Instead, we'll use GLPK as our solver and pulp as the Python API. I've included instructions for installing both libraries for Linux at the bottom of this post.

In [465]:
from pulp import *
import numpy as np

In [484]:
# Assume 10 collars.
# Let's make up some numbers


# Profit
p = [5.30, 8.23, 5.70, 3.32, 10.00, 5.00, 7.25, 3.90, 7.05, 6.56]

# Cost
w = [13.00, 19.30, 13.75, 8.54, 24.34, 13.90, 16.51, 9.00, 16.55, 14.10]

# Min run size
r = [50, 30, 25, 55, 70, 40, 40, 60, 30, 60]

sum([x*y for x,y in zip(r, w)])

6845.15

In [485]:
prob = LpProblem('Dog Collar Problem', LpMaximize)

In [486]:
budget = 10000.0
# Production counts (the variable in our problem)
c = []

for j in range(len(p)):
    max_count = np.floor(budget / w[j])
    c.append(LpVariable('c_{}'.format(j),
                        lowBound=0,
                        upBound=max_count,
                        cat='Integer'))

In [487]:
# Add objective function first
prob += lpSum([i * j for i, j in zip(p, c)]), 'Total profit'

# Budget constraint
prob += lpSum([i * j] for i, j in zip(w, c)) <= budget, 'Budget'

# Min batch size constraint
for j, (cnt, run) in enumerate(zip(c, r)):
    prob += cnt >= run, 'MinBatchSize_{}'.format(j)

# Run size difference
for i, cnt1 in enumerate(c[:-1]):
    for cnt2 in enumerate(c[i+1:]):
        prob += cnt1 - cnt2 <= 30
        prob += cnt2 - cnt1 <= 30
    

In [488]:
prob.writeLP('CollarModel.lp')

In [489]:
prob.solve()

1

In [490]:
print("Status:", LpStatus[prob.status])

('Status:', 'Optimal')


In [491]:
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

('c_0', '=', 53.0)
('c_1', '=', 78.0)
('c_2', '=', 51.0)
('c_3', '=', 55.0)
('c_4', '=', 70.0)
('c_5', '=', 48.0)
('c_6', '=', 78.0)
('c_7', '=', 77.0)
('c_8', '=', 74.0)
('c_9', '=', 75.0)


In [498]:
# The optimised objective function value is printed to the screen
print('Total profit = ${:6.2f}'.format(value(prob.objective)))

Total profit = $4215.64


#### PERSONAL INSTALLATION NOTES
Install pulp.
```bash
conda install --channel https://conda.anaconda.org/timcera pulp
```
Does not work with python 3
Install GLPK (it's free!).
```bash
sudo apt-get install glpk-utils
```
Test the installation. Open python, type
```python
import pulp
pulp.pulpTestAll()
```
Should see
```bash
	 Testing zero subtraction
	 Testing inconsistant lp solution
	 Testing continuous LP solution
	 Testing maximize continuous LP solution
	 Testing unbounded continuous LP solution
	 Testing Long Names
	 Testing repeated Names
	 Testing zero constraint
	 Testing zero objective
	 Testing LpVariable (not LpAffineExpression) objective
	 Testing Long lines in LP
	 Testing LpAffineExpression divide
	 Testing MIP solution
	 Testing MIP solution with floats in objective
	 Testing MIP relaxation
	 Testing feasibility problem (no objective)
	 Testing an infeasible problem
	 Testing an integer infeasible problem
	 Testing column based modelling
	 Testing dual variables and slacks reporting
	 Testing fractional constraints
	 Testing elastic constraints (no change)
	 Testing elastic constraints (freebound)
	 Testing elastic constraints (penalty unchanged)
	 Testing elastic constraints (penalty unbounded)
* Solver pulp.solvers.PULP_CBC_CMD passed.
Solver pulp.solvers.CPLEX_DLL unavailable
Solver pulp.solvers.CPLEX_CMD unavailable
Solver pulp.solvers.CPLEX_PY unavailable
Solver pulp.solvers.COIN_CMD unavailable
Solver pulp.solvers.COINMP_DLL unavailable
	 Testing zero subtraction
	 Testing inconsistant lp solution
	 Testing continuous LP solution
	 Testing maximize continuous LP solution
	 Testing unbounded continuous LP solution
	 Testing Long Names
	 Testing repeated Names
	 Testing zero constraint
	 Testing zero objective
	 Testing LpVariable (not LpAffineExpression) objective
	 Testing LpAffineExpression divide
	 Testing MIP solution
	 Testing MIP solution with floats in objective
	 Testing MIP relaxation
	 Testing feasibility problem (no objective)
	 Testing an infeasible problem
	 Testing an integer infeasible problem
	 Testing column based modelling
	 Testing fractional constraints
	 Testing elastic constraints (no change)
	 Testing elastic constraints (freebound)
	 Testing elastic constraints (penalty unchanged)
	 Testing elastic constraints (penalty unbounded)
* Solver pulp.solvers.GLPK_CMD passed.
Solver pulp.solvers.XPRESS unavailable
Solver pulp.solvers.GUROBI unavailable
Solver pulp.solvers.GUROBI_CMD unavailable
Solver pulp.solvers.PYGLPK unavailable
Solver pulp.solvers.YAPOSIB unavailable
```